####Initate Packages

In [ ]:

import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from nltk.probability import FreqDist
nltk.download('punkt')

  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

####Mount Google Drive (So you can get Data from the ML folder)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


####Import Data and Set up Data

In [ ]:
data = pd.read_csv(r'/content/drive/MyDrive/Machine Learning/Data/SDoutput1.csv')
#data = pd.read_csv('SDoutput1.csv')

In [ ]:
from ast import literal_eval
data['clean'] = data['clean'].apply(literal_eval)

data.loc[data["label"]=='suicide', "label"] = 1
data.loc[data["label"]=='non-suicide', "label"] = 0


print(data.head())
print(len(data))
print(len(data[data["label"] == 1]))
print(len(data[data["label"] == 0]))


   Unnamed: 0                                              clean label
0           0  [ex, wife, threaten, suiciderec, left, wife, g...     1
1           1  [weird, get, affect, compliment, come, someon,...     0
2           2  [final, 2020, almost, never, hear, 2020, ha, b...     0
3           3                  [need, helpjust, help, cri, hard]     1
4           4  [losthello, name, adam, 16, struggl, year, afr...     1
166612
83286
83326


##### Split data into suicidal and non suicidal tables

In [ ]:
dataSuicidal = data[data["label"] == 1]
dataNon = data[data["label"] == 0]


####Create frequency dist of words

In [ ]:
from nltk.probability import FreqDist

dataSuicidal_Freq = FreqDist()

for row in dataSuicidal['clean']:
  for word in row:
    dataSuicidal_Freq[word.lower()]+= 1
print(repr(dataSuicidal_Freq))

dataNon_Freq = FreqDist()

for row in dataNon['clean']:
  for word in row:
    dataNon_Freq[word.lower()]+= 1
print(repr(dataNon_Freq))


FreqDist({'thi': 131865, 'wa': 129270, 'want': 120297, 'feel': 107369, 'like': 98516, 'know': 87109, 'go': 86238, 'get': 81907, 'life': 79803, 'becaus': 69599, ...})
FreqDist({'like': 39092, 'thi': 38951, 'wa': 34612, 'filler': 31363, 'want': 25752, 'go': 23374, 'get': 23067, 'know': 22993, 'fuck': 20988, 'becaus': 17430, ...})


####Convert to freq table


In [ ]:
dataSuicidal_Ftable = pd.DataFrame.from_dict(dataSuicidal_Freq, orient = 'index')
dataSuicidal_Ftable.columns = ['Frequency Suicidal']
# dataSuicidal_Ftable

dataNon_Ftable = pd.DataFrame.from_dict(dataNon_Freq, orient = 'index')
dataNon_Ftable.columns = ['Frequency Non']
# dataNon_Ftable

Both_Ftable = pd.concat([dataSuicidal_Ftable, dataNon_Ftable], axis = 1)
Both_Ftable = Both_Ftable.fillna(0)

Both_Ftable.head(100000000)

,Frequency Suicidal,Frequency Non
ex,3409.0,505.0
wife,2692.0,250.0
threaten,1345.0,225.0
suiciderec,1.0,0.0
left,12157.0,1942.0
...,...,...
followtini,0.0,1.0
yearssss,0.0,1.0
happensth,0.0,1.0
66142,0.0,1.0


####TD-IDF Vectorization

In [ ]:
data_x = data["clean"]
data_y = data["label"]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state=100) #What does Random state do? Preset Rando?
print("Text training data before TD IDF")
print(x_train.head(),"\n")

def dummy_tokenizer(data):
  return data

cv = TfidfVectorizer(
    min_df = 1,
    analyzer='word',
    tokenizer=dummy_tokenizer,
    preprocessor=dummy_tokenizer,
    token_pattern = None)

x_traincv = cv.fit_transform(x_train)
x_testcv = cv.transform(x_test)
print("Text training data after TD IDF")
print(x_traincv)

Text training data before TD IDF
119342                      [anyon, chati, need, big, time]
9100      [realiz, scarecrow, call, scarecrow, becaus, s...
122475    [tonight, nightim, worthless, 24, almost, 25, ...
44869     [one, day, go, deadit, 's, hard, think, start,...
46426     [wonder, ever, think, time, seem, like, alway,...
Name: clean, dtype: object 

Text training data after TD IDF
  (0, 108790)	0.19532035560997257
  (0, 20938)	0.36512246997817227
  (0, 75498)	0.2324539365157479
  (0, 26134)	0.8451019515359366
  (0, 15909)	0.2455675049959109
  (1, 30866)	0.4042427873925088
  (1, 94493)	0.15419198311020513
  (1, 19861)	0.09229070860124082
  (1, 24488)	0.14033782184363078
  (1, 94497)	0.8695595880527512
  (1, 89616)	0.16872001919533117
  (2, 14335)	0.0915642263405557
  (2, 49648)	0.0372930651239029
  (2, 98767)	0.06661086430054902
  (2, 41201)	0.11785199109346173
  (2, 39601)	0.061781302682819846
  (2, 41719)	0.05483562743749304
  (2, 115038)	0.13314607322581512
  (2, 47349)	0.0

####MultinomialNB


In [ ]:

y_train = y_train.astype('int')
print("Training Data Labels")
print(y_train.head(4), "\n")

classifier = naive_bayes.MultinomialNB()
classifier.fit(x_traincv, y_train)

y_predicted = classifier.predict(x_testcv)
from sklearn.metrics import accuracy_score

print(y_predicted)
print(y_test)

accuracy = accuracy_score(y_test.tolist(), y_predicted)
print("\nAccuracy: ", accuracy)

Training Data Labels
119342    1
9100      0
122475    1
44869     1
Name: label, dtype: int64 

[0 1 1 ... 0 1 1]
150391    0
90086     1
112748    1
33856     1
119799    0
         ..
45086     0
35597     0
84634     0
122314    1
29851     1
Name: label, Length: 33323, dtype: object

Accuracy:  0.8809230861567086
